In [ ]:
!pip install tensornetwork

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.3/364.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Load and preprocess the MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784).astype(np.float32) / 255.0  # Normalize and ensure float32
x_test = x_test.reshape(-1, 784).astype(np.float32) / 255.0  # Normalize and ensure float32

# Define parameters
N = 784  # Number of pixels
d = 2  # Feature map dimension
D = 10  # Number of classes
bond_dim = 5  # Smaller bond dimension for simplicity

# Define a basic feature map (maps pixel values to a 2D vector)
def feature_map(x):
    return tf.stack([1.0 - x, x], axis=2)  # shape: [batch_size, 784, 2]

# Initialize MPS tensors with small dimensions
A = []
for i in range(N):
    if i == 0:
        A.append(tf.Variable(tf.random.normal([d, D, bond_dim], stddev=0.1, dtype=tf.float32)))
    elif i == N - 1:
        A.append(tf.Variable(tf.random.normal([bond_dim, d, D], stddev=0.1, dtype=tf.float32)))
    else:
        A.append(tf.Variable(tf.random.normal([bond_dim, d, bond_dim], stddev=0.1, dtype=tf.float32)))

# Define the simplest forward pass
def mps_forward(x):
    x = feature_map(x)  # shape: [batch_size, 784, 2]

    # First contraction (reduce to [batch_size, D, bond_dim])
    tensor = tf.tensordot(x[:, 0, :], A[0], axes=[[1], [0]])  # Now shape is [batch_size, D, bond_dim]

    # Iterate over the remaining pixels with very simple tensor multiplications
    for i in range(1, N - 1):
        tensor = tf.tensordot(tensor, A[i], axes=[[2], [2]])  # Now shape is [batch_size, bond_dim, bond_dim]

    # Final contraction to get the output logits
    output = tf.tensordot(tensor, A[-1], axes=[[2], [0]])  # Final shape should be [batch_size, D]

    return output

# Run a forward pass on the first 10 images
x_sample = x_train[:10]  # Take the first 10 images in the training set
output = mps_forward(x_sample)

# Print the output logits
print("Output shape:", output.shape)
print("Output logits:", output.numpy())


InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [1000,2], In[1]: [5,10] [Op:MatMul] name: 